# Malifaux Scapper

## Initialization

In [2]:
import tabulate
import requests as r
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def check_retailer(url, i):
    try:
        raw_page = r.get(f"{url}{i}").content
    except:
        raw_page = r.get(f"{url}{i}", verify=False).content
    soup = bs(raw_page, "html.parser")
    return soup

## Functions

In [3]:
def find_items(url, range_max, product_tag, name_tag, availability_tag, availability_string):
    last_page_contents = None
    available_products_list = set()
    for i in range(1,range_max):
        soup = check_retailer(url, i)
        products = soup.find_all(class_=product_tag)
        if products == last_page_contents or products == None: break
        for product in products:
            name = product.find(class_=name_tag).text
            try:
                availability = availability_string in product.find(class_=availability_tag).text
            except AttributeError:
                continue
            if availability:
                available_products_list.add(name.strip())
            print('Page:', i, end='\r')
        last_page_contents = products
    return available_products_list

In [4]:
def find_items_szopgracz(url, product_tag, name_tag, availability_tag, availability_string):
    available_products_list = set()
    soup = check_retailer(url, '')
    products = soup.find_all(class_=product_tag)
    for product in products:
        name = product.find(class_=name_tag).text
        availability = bool(product.find(class_=availability_tag))
        if not availability:
            available_products_list.add(name.strip())
    return available_products_list

In [5]:
def find_items_allegro():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://allegro.pl/kategoria/bitewne-inne-systemy-6059?string=malifaux&p=1")
    
    html = driver.page_source
    soup = bs(html, "html.parser")
    product_list = []
    for i in range(1,30):
        new_products = 0
        driver.get(f"https://allegro.pl/kategoria/bitewne-inne-systemy-6059?string=malifaux&p={i}")
        html = driver.page_source
        soup = bs(html, "html.parser")
        products = soup.find_all(class_='mgn2_14 m9qz_yp meqh_en mpof_z0 mqu1_16 m6ax_n4 mp4t_0 m3h2_0 mryx_0 munh_0 mj7a_4')
        for product in products:
            name = product.text
            if name not in product_list:
                print(name)
                product_list.append(name)
                new_products += 1 
            product_list.append(name)
        print('Page:', i, end='\r')
        if new_products == 0: break
    
    return product_list

In [ ]:
def generate_table(item_list):
    table_rows = []
    for item in item_list:
        table_row = []
        for shop in shops_dict:
            for product in shops_dict[shop]:
                if item.upper() in product.upper():
                    table_row.append('X')
                    break
            else:
                if item == '---------------------':
                    table_row.append('-'*len(shop))
                else:
                    table_row.append('-')
        table_rows.append([item.title(), *table_row])
    print(tabulate.tabulate(table_rows, headers=['box', *shops_dict.keys()], tablefmt="pretty"))

## Dictionary generation

In [6]:
shops_dict = {}

In [7]:
graal = find_items('https://sklep-graal.pl/pl/c/Malifaux-3rd/474/', 30, 'product-inner-wrap', 'productname', 'addtobasket', 'Do koszyka')
shops_dict['graal'] = graal


In [8]:
bolter = find_items('https://bolter.pl/pl/c/MALIFAUX-3ed./1211/', 30, 'product-inner-wrap', 'productname', 'addtobasket', 'Do koszyka')
shops_dict['bolter'] = bolter

In [9]:
strefamtg = find_items('https://strefamtg.pl/2596-malifaux-3ed?page=', 30, 'product-miniature__container', 'product-miniature__title', 'product-quantities', 'Dostępne')
shops_dict['strefa'] = strefamtg

In [10]:
szopgracz = find_items_szopgracz('https://shopgracz.pl/298-malifaux', 'item-product', 'product_name', 's_soon', 'niedostępny')
shops_dict['szop'] = szopgracz

In [11]:
# shadowarrior = find_items('https://shadowarrior.pl/pl/c/MALIFAUX-WYRD/132/', 30, 'product-inner-wrap', 'productname', 'addtobasket', 'Do koszyka')

In [12]:
gnom = find_items('https://gnom-sklep.pl/447-malifaux-3rd-edition?page=', 30, 'product-miniature', 'h3', 'hiaddtocart', 'Dostępny')
shops_dict['gnom'] = gnom

c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\asmialek\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gnom-sklep.pl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
allegro = find_items_allegro()
shops_dict['allegro'] = allegro

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\asmialek\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\asmialek\AppData\Local\Temp\ipykernel_18724\2381978508.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Malifaux 3rd - Sandeep Core Box
Malifaux 3rd Edition - Sandeep Core Box - EN
Zestaw Figurek Malifaux 3E Embrace the Ember
Malifaux 3E Barbaros
Malifaux 3E Here Lies…
Malifaux 3E Sand Worm
MALIFAUX 3E - WYR23213 Rogue Necromancy
Malifaux 3E Crime and Punishment
Malifaux 3E Pigapult
Malifaux 3rd Edition - Rank and File - EN
Malifaux 3E Basse Core Box
Malifaux 3E Heavy Metal
Malifaux 3E Asami Core Box
Malifaux 3E Maxine Core Box
Malifaux 3E Rotten Harvest Toil and Trouble
MALIFAUX 3E - WYR23110 From Above
Malifaux 3E Von Schtook Core Box
Malifaux 3rd - Nellie Core Box
Malifaux 3E Chained Magic
MALIFAUX 3E - WYR23519 Hamelin Core Box
MALIFAUX 3E - WYR23714 Mei Feng Core Box
Malifaux M3e: Basse Core Box
Malifaux 3rd Edition - Leveticus Core Box
Malifaux 3rd Edition - Hoffman Core Box - EN
Malifaux 3rd Edition - Euripides Core Box - EN
Malifaux 3rd Edition - Kirai Core Box - EN
Malifaux 3E Internal Investigation
Malifaux 3rd Edition - Anya Core Box
MALIFAUX 3E - WYR23210 Stitched And Sewn
Ma

## Dict check

In [14]:
shops_dict

{'graal': {'Alt Mr. Graves and Mr. Tannen',
  'Alt Nekima',
  'Ancestral Icons',
  'Bayou Faction Pack',
  'Duality Fate Deck',
  'Embrace the Ember',
  'Leveticus Core Box',
  'Malifaux : Botanists',
  'Malifaux : Gaining Grounds Season 1 Pack',
  'Malifaux : Lacroix Raiders Special Order',
  'Malifaux : Rotten Harvest Beware The Lights',
  'Malifaux : The Tortoise and The Hare',
  'Malifaux : Toil and Trouble Rotten Harvest',
  'Malifaux:  Iconic - Scorch the Soul',
  'Malifaux:  Trappers',
  'Malifaux: A Light in the Dark',
  'Malifaux: Agent 46',
  'Malifaux: Akaname',
  'Malifaux: Alan Reid and the False Witnesses',
  'Malifaux: Alphonse LeBlanc',
  'Malifaux: Anya Core Box',
  'Malifaux: Arcane Fate',
  'Malifaux: Arcanist Faction Book',
  'Malifaux: Arcanist Fate Deck',
  'Malifaux: Asami Core Box',
  'Malifaux: Ashes and Dust',
  'Malifaux: Auxiliary Forces',
  'Malifaux: Aversion x3',
  'Malifaux: Barbaros',
  'Malifaux: Basse Core Box',
  'Malifaux: Bayou Faction Book',
  'Ma

## Table generation

In [22]:
item_list = ['Tara Core Box',
             'Servants of the Void',
             'Beyond Time',
             '---------------------',
             'KAERIS CORE BOX',
             'BURNING BRIDGES', 
             '---------------------',
             'LADY JUSTICE CORE BOX',
             'WAKE THE DEAD',
             '---------------------',
             'SONNIA CORE BOX',
             'WITCHING HOUR',
             '---------------------',
             'SANDEEP CORE BOX',
             'PEER REVIEW',
             '---------------------',
             'VON SCHILL CORE BOX',
             'BETWEEN THE LEY LINES',
             '---------------------',
             'Misaki Core Box',
             'CRIME SYNDICATE',
             'Silent Strike',
             'Dawn Serpent',
             'Dawn Sepent',
             'Kharmic Debt',
             'Karmic Debt',
             '---------------------',
             'ASAMI CORE BOX',
             'ANCIENT EVIL',
             'Immortal Tricksters',
             'Akaname',
             '---------------------',
             'JEDZA CORE BOX',
             'HERE LIES...',
             'HERE LIES',
             ]
generate_table(item_list)        

+-----------------------+-------+--------+--------+------+------+---------+
|          box          | graal | bolter | strefa | szop | gnom | allegro |
+-----------------------+-------+--------+--------+------+------+---------+
|     Tara Core Box     |   -   |   -    |   -    |  -   |  -   |    -    |
| Servants Of The Void  |   -   |   -    |   -    |  -   |  -   |    -    |
|      Beyond Time      |   X   |   -    |   -    |  -   |  -   |    -    |
| --------------------- | ----- | ------ | ------ | ---- | ---- | ------- |
|    Kaeris Core Box    |   X   |   -    |   -    |  -   |  -   |    X    |
|    Burning Bridges    |   X   |   -    |   -    |  -   |  -   |    -    |
| --------------------- | ----- | ------ | ------ | ---- | ---- | ------- |
| Lady Justice Core Box |   -   |   -    |   -    |  -   |  -   |    -    |
|     Wake The Dead     |   -   |   -    |   -    |  -   |  -   |    -    |
| --------------------- | ----- | ------ | ------ | ---- | ---- | ------- |
|    Sonnia 